# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [24]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json

# Import API key
from api_keys import geoapify_key

In [25]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,atafu village,-8.5421,-172.5159,28.83,76,99,6.28,TK,1686528382
1,1,saldanha,-33.0117,17.9442,14.69,86,88,5.26,ZA,1686528493
2,2,ziro,27.6333,93.8333,14.97,99,99,0.21,IN,1686528493
3,3,olonkinbyen,70.9221,-8.7187,1.08,90,100,5.65,SJ,1686528494
4,4,whitehorse,60.7161,-135.0538,14.45,38,100,8.23,CA,1686528350


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [26]:
%%capture --no-display

# Configure the map plot
map_plot_all = city_data_df.hvplot.points("Lng","Lat", geo=True, tiles="OSM",
                                      size="Humidity",frame_height=500, 
                                      frame_width=700, color="City", alpha=0.50)


# Display the map
map_plot_all

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [27]:
# Narrow down cities that fit criteria and drop any results with null values

# Temp_ideal_cities = city_data_df.loc[city_data_df["Max Temp"]>21,:].loc[city_data_df["Max Temp"]<27,:]
# Wind_ideal_cities = Temp_ideal_cities.loc[city_data_df["Wind Speed"]<4.5,:]
# cloudiness_ideal_cities = Wind_ideal_cities.loc[Wind_ideal_cities["Cloudiness"]==0,:]

# combine all the search from above into a single line 
Vacation_ideal_cities = (city_data_df.loc[city_data_df["Max Temp"]>21,:].
                         loc[city_data_df["Max Temp"]<27,:].
                         loc[city_data_df["Wind Speed"]<4.5,:].
                         loc[city_data_df["Cloudiness"]==0,:])

# Drop any rows with null values
Vacation_ideal_cities = Vacation_ideal_cities.dropna()
Vacation_ideal_cities

# Display sample data
Vacation_ideal_cities

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
8,8,bethel,41.3712,-73.4140,23.26,63,0,3.09,US,1686528495
55,55,tazacorte,28.6290,-17.9293,22.54,84,0,3.13,ES,1686528505
67,67,saint-pierre,-21.3393,55.4781,21.82,68,0,2.57,RE,1686528508
91,91,petrolina,-9.3986,-40.5008,25.03,73,0,3.09,BR,1686528514
119,119,urumqi,43.8010,87.6005,23.53,36,0,3.00,CN,1686528520
305,305,colonia,50.9333,6.9500,21.03,51,0,3.09,DE,1686527858
395,395,detroit lakes,46.8172,-95.8453,22.03,30,0,4.12,US,1686528584
520,520,international falls,48.6011,-93.4110,23.10,40,0,2.57,US,1686528617
557,557,antonio enes,-16.2325,39.9086,22.44,83,0,2.95,MZ,1686528625


### Step 3: Create a new DataFrame called `hotel_df`.

In [29]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = Vacation_ideal_cities[["City","Country","Lat","Lng","Humidity"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""


# Display sample data
hotel_df

C:\Users\faraz\AppData\Local\Temp\ipykernel_66368\2692859917.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = ""


,City,Country,Lat,Lng,Humidity,Hotel Name
8,bethel,US,41.3712,-73.4140,63,
55,tazacorte,ES,28.6290,-17.9293,84,
67,saint-pierre,RE,-21.3393,55.4781,68,
91,petrolina,BR,-9.3986,-40.5008,73,
119,urumqi,CN,43.8010,87.6005,36,
305,colonia,DE,50.9333,6.9500,51,
395,detroit lakes,US,46.8172,-95.8453,30,
520,international falls,US,48.6011,-93.4110,40,
557,antonio enes,MZ,-16.2325,39.9086,83,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [30]:
# Set parameters to search for a hotel
radius = 10000
params = {"categories": "accommodation.hotel",
          "format":"json",
          "apiKey":geoapify_key,
          "limit":20}


# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
#     get latitude, longitude from the DataFrame
    Lat = row["Lat"]
    Lng = row["Lng"]
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{Lng},{Lat},{radius}"
    params["bias"] = f"proximity:{Lng},{Lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
#     Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df



Starting hotel search
bethel - nearest hotel: Hampton Inn Danbury
tazacorte - nearest hotel: App Leyma
saint-pierre - nearest hotel: Tropic Hotel
petrolina - nearest hotel: Neuman
urumqi - nearest hotel: 百时快捷酒店红山店
colonia - nearest hotel: Wasserturm Hotel Cologne
detroit lakes - nearest hotel: Graystone Hotel
international falls - nearest hotel: Copper River Inn & Conference Centre
antonio enes - nearest hotel: Hotel Quirimbas


,City,Country,Lat,Lng,Humidity,Hotel Name
8,bethel,US,41.3712,-73.4140,63,Hampton Inn Danbury
55,tazacorte,ES,28.6290,-17.9293,84,App Leyma
67,saint-pierre,RE,-21.3393,55.4781,68,Tropic Hotel
91,petrolina,BR,-9.3986,-40.5008,73,Neuman
119,urumqi,CN,43.8010,87.6005,36,百时快捷酒店红山店
305,colonia,DE,50.9333,6.9500,51,Wasserturm Hotel Cologne
395,detroit lakes,US,46.8172,-95.8453,30,Graystone Hotel
520,international falls,US,48.6011,-93.4110,40,Copper River Inn & Conference Centre
557,antonio enes,MZ,-16.2325,39.9086,83,Hotel Quirimbas


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [32]:
%%capture --no-display

# Configure the map plot
map_plot_vacation = hotel_df.hvplot.points("Lng","Lat", geo=True, tiles="OSM",
                                      size="Humidity",frame_height=500, 
                                      frame_width=700, color="City", alpha=0.50, hover_cols=['Hotel Name','Country'],
                                          )


# Display the map
map_plot_vacation

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)